In [1]:

from mindquantum.algorithm.nisq import HardwareEfficientAnsatz,RYFull
from mindquantum.core.parameterresolver import  PRGenerator
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, I,CNOT
from mindquantum.core.circuit import Circuit,UN
import mindspore as ms
from mindquantum.core.parameterresolver import PRGenerator
import random
from mindspore import Tensor,ops
import tensorcircuit as tc
import tensorflow as tf
import mindspore.numpy as mnp
from DQAS_tool import wash_pr,Mindspore_ansatz_micro,vag_nnp_micro
import sys
from typing import Union
sys.path.append('..')
from Test_tool import Test_ansatz
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.core.circuit import change_param_name,apply

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR
Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [5]:
pr_pool = PRGenerator('pool')
parameterized_circuit= \
[UN(RX(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RX(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RX(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RX(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),
 UN(RY(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RY(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RY(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RY(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),
 UN(RZ(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RZ(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RZ(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RZ(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),]


unparameterized_circuit = \
[UN(X,maps_obj=[0],maps_ctrl=[1]),
 UN(X,maps_obj=[1],maps_ctrl=[0]),
 UN(Z,maps_obj=[0],maps_ctrl=[1]),
 UN(Z,maps_obj=[1],maps_ctrl=[0]),
 UN(Y,maps_obj=[0],maps_ctrl=[1]),
 UN(Y,maps_obj=[1],maps_ctrl=[0]),
 ]
ansatz_pr = PRGenerator('ansatz')
shape_parametized = len(parameterized_circuit)
shape_unparameterized = len(unparameterized_circuit)
num_layer=10
shape_nnp = (7,num_layer,shape_parametized)
shape_stp = (num_layer,shape_unparameterized+shape_parametized)
stddev = 0.03
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)
ops_onehot = ops.OneHot(axis=-1)

In [3]:
def best_from_structure(structures: np.array)->Tensor:
    prob = ops.Softmax()(ms.Tensor(structures))
    return ops.Argmax(axis=-1)(prob)

def best_from_structure(structures: np.array)->Tensor:
    return ops.Argmax(axis=-1)(ms.Tensor(structures))

In [6]:
ops_onehot = ops.OneHot(axis=-1)
stp_onehot = best_from_structure(stp)
stp_onehot = ops_onehot(ms.Tensor(np.array([5,3,4,8,5,0,2,2,8,4])),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))



ansatz = Mindspore_ansatz_micro(Structure_p=stp_onehot,parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,num_layer=num_layer,n_qbits=8)


In [16]:
from mindquantum.simulator import Simulator
from mindquantum.core.gates import Measure
sim = Simulator('mqvector',n_qubits=8)
ansatz_measure = ansatz + Measure('q0').on(0)
sim.apply_circuit(ansatz_measure,pr=dict(zip(ansatz.params_name,np.random.randn(len(ansatz.params_name)))))

shots: 1
Keys: q0│0.00     0.2         0.4         0.6         0.8         1.0
────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
       1│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
        │
{'1': 1}

In [12]:
ansatz_measure

┏━━━━━━━━━━━━━━┓ ┏━━━┓                           ┏━━━━━━━━━━━━━┓            
q0: ──┨ RY(encoder0) ┠─┨ I ┠──────────────────■────────┨ RY(ansatz2) ┠─[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛ ┗━━━┛                  ┃        ┗━━━━━━━━━━━━━┛            
      ┏━━━━━━━━━━━━━━┓ ┏━━━━━━━━━━━━━┓ ┏━━━━━━┻━━━━━━┓ ┏━━━┓                      
q1: ──┨ RY(encoder1) ┠─┨ RY(ansatz0) ┠─┨ RX(ansatz1) ┠─┨ I ┠───────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛ ┗━━━━━━━━━━━━━┛ ┗━━━━━━━━━━━━━┛ ┗━━━┛                      
      ┏━━━━━━━━━━━━━━┓                                                            
q2: ──┨ RY(encoder2) ┠─────────────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛                                                            
      ┏━━━━━━━━━━━━━━┓                                                            
q3: ──┨ RY(encoder3) ┠─────────────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛                                                            
      ┏━━━━━━━━━━━━━━┓                                                            
q4: ──┨ RY(encoder4) ┠─────────────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛                                                            
      ┏━━━━━━━━━━━━━━┓                                                            
q5: ──┨ RY(encoder5) ┠─────────────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛                                                            
      ┏━━━━━━━━━━━━━━┓                                                            
q6: ──┨ RY(encoder6) ┠─────────────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛                                                            
      ┏━━━━━━━━━━━━━━┓                                                            
q7: ──┨ RY(encoder7) ┠─────────────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━━━━━━━━━━━━┛                                                            
                                                                                  
      ┏━━━━━━━━━━━━━┓ ┏━━━┓ ┏━━━━━━━━━━━━━┓ ┏━━━━━━━━━━━━━┓ ┏━━━━━━━━━━━━━┓       
q0: ──┨ RZ(ansatz3) ┠─┨ I ┠─┨ RX(ansatz5) ┠─┨ RX(ansatz6) ┠─┨ RX(ansatz7) ┠─[red bold]↯[/]─    
      ┗━━━━━━━━━━━━━┛ ┗━━━┛ ┗━━━━━━━━━━━━━┛ ┗━━━━━━┳━━━━━━┛ ┗━━━━━━┳━━━━━━┛       
      ┏━━━┓ ┏━━━━━━━━━━━━━┓ ┏━━━┓                  ┃               ┃              
q1: ──┨ I ┠─┨ RY(ansatz4) ┠─┨ I ┠──────────────────■───────────────■────────[red bold]↯[/]─    
      ┗━━━┛ ┗━━━━━━━━━━━━━┛ ┗━━━┛                                                 
                                                                                  
q2: ────────────────────────────────────────────────────────────────────────[red bold]↯[/]─    
                                                                                  
                                                                                  
q3: ────────────────────────────────────────────────────────────────────────[red bold]↯[/]─    
                                                                                  
                                                                                  
q4: ────────────────────────────────────────────────────────────────────────[red bold]↯[/]─    
                                                                                  
                                                                                  
q5: ────────────────────────────────────────────────────────────────────────[red bold]↯[/]─    
                                                                                  
                                                                                  
q6: ────────────────────────────────────────────────────────────────────────[red bold]↯[/]─    
                                                                            